In [ ]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine



file_path = 'path_to_data_file.csv'
df = pd.read_csv(file_path, delimiter='|')



df['Open_Date'] = pd.to_datetime(df['Open_Date'], format='%Y%m%d', errors='coerce')
df['Last_Consulted_Date'] = pd.to_datetime(df['Last_Consulted_Date'], format='%Y%m%d', errors='coerce')


df['DOB'] = pd.to_datetime(df['DOB'], format='%d%m%Y', errors='coerce')
df['Age'] = df['DOB'].apply(lambda x: (datetime.now() - x).days // 365 if pd.notnull(x) else None)


df['Days_Since_Last_Consulted'] = (datetime.now() - df['Last_Consulted_Date']).dt.days


df['Days_Consulted_Flag'] = df['Days_Since_Last_Consulted'].apply(lambda x: 'Yes' if x > 30 else 'No')


df = df.dropna(subset=['Customer_ID'])


df['Customer_Name'] = df['Customer_Name'].astype(str).str[:255]
df['Vaccination_Type'] = df['Vaccination_Type'].astype(str).str[:5]


df.to_sql('staging_customers', con=engine, if_exists='replace', index=False)


countries = df['Country'].unique()

for country in countries:
    country_df = df[df['Country'] == country]
    country_table = f"Customers_{country}"
    
  
    country_df.to_sql(country_table, con=engine, if_exists='replace', index=False)

print("Data successfully loaded into country-specific tables!")